In [ ]:
pip install plotly

In [ ]:
import plotly.graph_objects as go
import pandas as pd

nodes = pd.read_csv("
_nodes.csv")   # or use the DataFrame shown
links = pd.read_csv("sankey_links.csv")

fig = go.Figure(data=[go.Sankey(
        arrangement="snap",
        node=dict(
            pad=12,
            thickness=18,
            label=nodes["label"],
            hovertemplate="%{label}<extra></extra>"
        ),
        link=dict(
            source=links["source"],
            target=links["target"],
            value =links["value"],
            hovertemplate="%{value} incidents<extra></extra>"
        )
)])
fig.update_layout(title_text="Smart-Contract Incidents:  Vulnerability → Attack Vector → Loss Bracket")
fig.show()

In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Create a dataset based on the analysis of the appendix.
# Columns: ID, Vulnerabilities, Attack_Vectors, Loss_USD
data = {
    'ID': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50],
    'Vulnerabilities': [
        'Access Control;Business Logic', 'Access Control', 'Price Manipulation', 'Price Manipulation', 'Reentrancy',
        'Access Control', 'Business Logic', 'Reentrancy', 'Rounding Error', 'Price Manipulation', 'Access Control',
        'Access Control', 'Input Validation', 'Access Control', 'Access Control;Input Validation', 'Misconfiguration',
        'Rounding Error', 'Unchecked Call Return Value;Access Control', 'Price Manipulation;Rounding Error',
        'Business Logic;Rounding Error', 'Price Manipulation', 'Integer Underflow', 'Access Control', 'Price Manipulation',
        'Reentrancy', 'Price Manipulation;Access Control', 'Storage Collision', 'Business Logic', 'Input Validation',
        'Price Manipulation', 'Access Control;Input Validation', 'Input Validation', 'Reentrancy', 'Governance',
        'Input Validation', 'Rounding Error', 'Price Manipulation', 'Price Manipulation', 'Integer Underflow',
        'Access Control;Governance', 'Dangerous Delegatecall', 'Input Validation', 'Dangerous Delegatecall',
        'Price Manipulation', 'Reentrancy', 'Governance', 'Reentrancy;Price Manipulation', 'Price Manipulation;Access Control',
        'Access Control', 'Reentrancy'
    ],
    'Attack_Vectors': [
        'Flash Loan;Logic Abuse', 'Flawed Upgrade', 'Oracle Manipulation', 'Flash Loan;Logic Abuse', 'Flash Loan;Reentrancy Callback',
        'Privilege Abuse', 'Flash Loan;Logic Abuse', 'Flash Loan;Reentrancy Callback', 'Flash Loan;Logic Abuse', 'Oracle Manipulation',
        'Privilege Abuse', 'Logic Abuse', 'Logic Abuse', 'Flawed Upgrade', 'Logic Abuse', 'Logic Abuse',
        'Flash Loan;Logic Abuse', 'Logic Abuse', 'Flash Loan;Logic Abuse', 'Flash Loan;Logic Abuse', 'Oracle Manipulation',
        'Logic Abuse', 'Logic Abuse', 'Flash Loan;Logic Abuse', 'Reentrancy Callback;Oracle Manipulation', 'Oracle Manipulation',
        'Logic Abuse', 'Logic Abuse', 'Flash Loan;Logic Abuse', 'Flash Loan;Logic Abuse', 'Delegatecall', 'Flash Loan;Logic Abuse',
        'Reentrancy Callback', 'Governance Proposal', 'Logic Abuse', 'Flash Loan;Logic Abuse', 'Privilege Abuse;Oracle Manipulation',
        'Flash Loan;Oracle Manipulation', 'Flash Loan;Logic Abuse', 'Governance Proposal;Delegatecall', 'Delegatecall', 'Logic Abuse',
        'Privilege Abuse;Delegatecall', 'Flash Loan;Oracle Manipulation', 'Reentrancy Callback;Oracle Manipulation', 'Governance Proposal',
        'Flash Loan;Reentrancy Callback;Oracle Manipulation', 'Flash Loan;Logic Abuse', 'Flawed Upgrade', 'Flash Loan;Reentrancy Callback'
    ],
    'Loss_USD': [
        197000000, 190000000, 120000000, 85000000, 80000000, 49500000, 48000000, 27000000, 20000000, 15600000,
        14400000, 6500000, 6900000, 12000000, 11600000, 11400000, 9600000, 11600000, 7400000, 6500000,
        6500000, 5000000, 4900000, 4500000, 4200000, 7500000, 355000, 13000000, 4850000, 3800000, 2000000,
        2100000, 501279, 2173500, 15800000, 3600000, 7500000, 19400000, 6800000, 16000000, 2100000,
        6400000, 2700000, 2100000, 3400000, 2500000, 800000, 7500000, 3000000, 3000000
    ]
}
df = pd.DataFrame(data)

# Define Loss Brackets
def get_loss_bracket(loss):
    if loss > 50000000:
        return 'Major Loss (> $50M)'
    elif loss > 10000000:
        return 'High Loss ($10M - $50M)'
    elif loss > 1000000:
        return 'Medium Loss ($1M - $10M)'
    else:
        return 'Low Loss (< $1M)'
df['Loss_Bracket'] = df['Loss_USD'].apply(get_loss_bracket)

# Explode the dataframe to handle multiple vulnerabilities and vectors per incident
df_exp = df.assign(Vulnerabilities=df['Vulnerabilities'].str.split(';'),
                   Attack_Vectors=df['Attack_Vectors'].str.split(';')).explode('Vulnerabilities').explode('Attack_Vectors')


# Simplify Categories for a cleaner Sankey Diagram
def simplify_vulnerability(vuln):
    if vuln in ['Access Control', 'Misconfiguration', 'Unchecked Call Return Value', 'Input Validation', 'Storage Collision']:
        return 'Access, Input & State'
    if vuln in ['Price Manipulation', 'Business Logic', 'Rounding Error', 'Integer Underflow']:
        return 'Price & Economic Logic'
    if vuln in ['Governance', 'Dangerous Delegatecall']:
        return 'Governance & Upgradability'
    if vuln in ['Reentrancy']:
        return 'Reentrancy'
    return 'Other'

def simplify_vector(vec):
    if vec in ['Flash Loan', 'Oracle Manipulation', 'Logic Abuse']:
        return 'Economic Exploit'
    if vec in ['Privilege Abuse', 'Flawed Upgrade']:
        return 'Permission & Upgrade Abuse'
    if vec in ['Governance Proposal']:
        return 'Governance Proposal'
    if vec in ['Reentrancy Callback']:
        return 'Reentrancy Callback'
    if vec in ['Delegatecall']:
        return 'Delegatecall'
    return 'Other'

df_exp['Vulnerability_Group'] = df_exp['Vulnerabilities'].apply(simplify_vulnerability)
df_exp['Attack_Vector_Group'] = df_exp['Attack_Vectors'].apply(simplify_vector)

# Create links for the Sankey diagram
# Flow 1: Vulnerability -> Attack Vector
flow1 = df_exp.groupby(['Vulnerability_Group', 'Attack_Vector_Group']).size().reset_index(name='value')
flow1.columns = ['source', 'target', 'value']

# Flow 2: Attack Vector -> Loss Bracket
flow2 = df_exp.groupby(['Attack_Vector_Group', 'Loss_Bracket']).size().reset_index(name='value')
flow2.columns = ['source', 'target', 'value']

# Combine flows
all_links = pd.concat([flow1, flow2], axis=0)

# Create the list of unique nodes
unique_nodes = pd.unique(all_links[['source', 'target']].values.ravel('K'))
node_map = {node: i for i, node in enumerate(unique_nodes)}

# Map sources and targets to numerical indices
all_links['source'] = all_links['source'].map(node_map)
all_links['target'] = all_links['target'].map(node_map)

# Define node colors - assigning colors to each group for better visualization
colors = {
    'Access, Input & State': '#1f77b4', # Muted Blue
    'Price & Economic Logic': '#ff7f0e', # Safety Orange
    'Reentrancy': '#d62728', # Brick Red
    'Governance & Upgradability': '#9467bd', # Muted Purple
    'Economic Exploit': '#2ca02c', # Cooked Asparagus Green
    'Permission & Upgrade Abuse': '#8c564b', # Chestnut
    'Reentrancy Callback': '#e377c2', # Razzle Dazzle Rose
    'Governance Proposal': '#7f7f7f', # Middle Gray
    'Delegatecall': '#bcbd22', # Murky Yellow
    'Major Loss (> $50M)': '#EF553B', # Red
    'High Loss ($10M - $50M)': '#FECB52', # Yellow
    'Medium Loss ($1M - $10M)': '#00CC96', # Green
    'Low Loss (< $1M)': '#636EFA' # Blue
}
node_colors = [colors.get(node, '#A9A9A9') for node in unique_nodes]


# Create the Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=25,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=unique_nodes,
        color=node_colors
    ),
    link=dict(
        source=all_links['source'],
        target=all_links['target'],
        value=all_links['value']
    ))])

fig.update_layout(title_text="Flow of Vulnerabilities to Financial Loss in Smart Contract Hacks",
                  font_size=12,
                  width=1000,
                  height=700)
fig.show()


In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Data extracted from the appendix, focusing on multi-vulnerability incidents
attack_chains_data = {
    'chain': [
        'Access Control->Business Logic Flaw',
        'Access Control->Input Validation',
        'Access Control->Input Validation',
        'Access Control->Price Manipulation',
        'Access Control->Price Manipulation',
        'Business Logic Flaw->Rounding Error',
        'Reentrancy->Price Manipulation',
        'Reentrancy->Price Manipulation',
        'Reentrancy->Price Manipulation',
        'Price Manipulation->Rounding Error',
        'Price Manipulation->Input Validation',
        'Input Validation->Dangerous Delegatecall',
        'Governance->Dangerous Delegatecall',
        'Private Key Compromise->Price Manipulation',
        'Private Key Compromise->Dangerous Delegatecall'
    ],
    'loss_usd': [
        197000000,  # Euler Finance
        6900000,    # Ionic Money (simplified chain)
        11600000,   # LI.FI
        7500000,    # KiloEx
        7500000,    # Jimbo's Protocol
        6500000,    # Abracadabra Money
        4200000,    # Conic Finance
        3400000,    # EraLend
        800000,     # Sturdy Finance
        7400000,    # Hundred Finance
        3800000,    # Onyx Protocol
        2000000,    # Dexible
        16000000,   # Curio
        7500000,    # Rho Market
        2700000     # OKX DEX
    ]
}

df = pd.DataFrame(attack_chains_data)

# Split chain into source and target vulnerabilities
df[['vuln1', 'vuln2']] = df['chain'].str.split('->', expand=True)

# Define Loss Brackets
def get_loss_bracket(loss):
    if loss > 50000000:
        return 'Major Loss (> $50M)'
    elif loss > 10000000:
        return 'High Loss ($10M - $50M)'
    elif loss > 1000000:
        return 'Medium Loss ($1M - $10M)'
    else:
        return 'Low Loss (< $1M)'
df['loss_bracket'] = df['loss_usd'].apply(get_loss_bracket)

# Prepare links for the Sankey diagram
# Flow 1: Vulnerability 1 -> Vulnerability 2
flow1 = df.groupby(['vuln1', 'vuln2']).size().reset_index(name='value')
flow1.columns = ['source', 'target', 'value']

# Flow 2: Vulnerability 2 -> Loss Bracket
flow2 = df.groupby(['vuln2', 'loss_bracket']).size().reset_index(name='value')
flow2.columns = ['source', 'target', 'value']

# Combine flows
all_links = pd.concat([flow1, flow2], axis=0)

# Create the list of unique nodes and map them to indices
unique_nodes = pd.unique(all_links[['source', 'target']].values.ravel('K'))
node_map = {node: i for i, node in enumerate(unique_nodes)}

all_links['source'] = all_links['source'].map(node_map)
all_links['target'] = all_links['target'].map(node_map)

# Define node colors for clarity
node_colors = [
    '#1f77b4', '#aec7e8', '#ff7f0e', '#ffbb78', '#2ca02c', '#98df8a',
    '#d62728', '#ff9896', '#9467bd', '#c5b0d5', '#8c564b', '#c49c94',
    '#e377c2', '#f7b6d2', '#7f7f7f', '#c7c7c7'
]
# Assign specific colors to loss brackets for emphasis
color_map = {
    'Major Loss (> $50M)': '#d62728',
    'High Loss ($10M - $50M)': '#ff7f0e',
    'Medium Loss ($1M - $10M)': '#2ca02c',
    'Low Loss (< $1M)': '#1f77b4'
}
plot_node_colors = [color_map.get(node, '#A9A9A9') for node in unique_nodes]


# Create the Sankey diagram
fig = go.Figure(data=[go.Sankey(
    arrangement='snap',
    node=dict(
        pad=25,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=unique_nodes,
        color=plot_node_colors
    ),
    link=dict(
        source=all_links['source'],
        target=all_links['target'],
        value=all_links['value']
    ))])

fig.update_layout(
    title_text="Exploit Chains: Flow of Multi-Vulnerability Attacks to Financial Loss",
    font_size=12,
    width=1000,
    height=700
)

fig.show()

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.colors

# Manually compiled data of multi-vulnerability attack chains, now including incident names for the legend.
attack_chains_data = [
    {'chain_name': 'Euler Finance', 'vuln_entry': 'Access Control', 'vuln_payload': 'Business Logic', 'count': 1, 'loss': 197000000},
    {'chain_name': 'LI.FI, Dexible', 'vuln_entry': 'Input Validation', 'vuln_payload': 'Unsafe Call/Delegatecall', 'count': 3, 'loss': 11600000 * 2 + 2000000},
    {'chain_name': 'Hundred, Abracadabra', 'vuln_entry': 'Price/Logic', 'vuln_payload': 'Rounding Error', 'count': 2, 'loss': 7400000 + 6500000},
    {'chain_name': 'Conic, EraLend, Sturdy', 'vuln_entry': 'Reentrancy', 'vuln_payload': 'Price Manipulation', 'count': 3, 'loss': 4200000 + 3400000 + 800000},
    {'chain_name': 'KiloEx, Jimbo\'s', 'vuln_entry': 'Access Control', 'vuln_payload': 'Price Manipulation', 'count': 2, 'loss': 7500000 * 2},
    {'chain_name': 'Onyx Protocol', 'vuln_entry': 'Price Manipulation', 'vuln_payload': 'Input Validation', 'count': 1, 'loss': 3800000},
    {'chain_name': 'Curio, OKX DEX', 'vuln_entry': 'Governance', 'vuln_payload': 'Dangerous Delegatecall', 'count': 2, 'loss': 16000000 + 2700000},
    {'chain_name': 'Rho Market', 'vuln_entry': 'Privilege Abuse', 'vuln_payload': 'Price Manipulation', 'count': 1, 'loss': 7500000},
]

df_chains = pd.DataFrame(attack_chains_data)

# Define Loss Brackets
def get_loss_bracket(loss, count):
    avg_loss = loss / count
    if avg_loss > 50000000:
        return 'Major Loss (> $50M)'
    elif avg_loss > 10000000:
        return 'High Loss ($10M - $50M)'
    elif avg_loss > 1000000:
        return 'Medium Loss ($1M - $10M)'
    else:
        return 'Low Loss (< $1M)'

df_chains['Loss_Bracket'] = df_chains.apply(lambda row: get_loss_bracket(row['loss'], row['count']), axis=1)

# --- Prepare data for Sankey ---
# Define a color palette
color_palette = plotly.colors.qualitative.Plotly
df_chains['color'] = [color_palette[i % len(color_palette)] for i in range(len(df_chains))]

# Create a list of all nodes
all_nodes = pd.concat([df_chains['vuln_entry'], df_chains['vuln_payload'], df_chains['Loss_Bracket']]).unique()
node_map = {node: i for i, node in enumerate(all_nodes)}

# Create links and link_colors
links = {'source': [], 'target': [], 'value': [], 'color': []}
for _, row in df_chains.iterrows():
    # Flow 1: Entry -> Payload
    links['source'].append(node_map[row['vuln_entry']])
    links['target'].append(node_map[row['vuln_payload']])
    links['value'].append(row['count'])
    links['color'].append(row['color'])
    
    # Flow 2: Payload -> Loss Bracket
    links['source'].append(node_map[row['vuln_payload']])
    links['target'].append(node_map[row['Loss_Bracket']])
    links['value'].append(row['count'])
    links['color'].append(row['color'])

# --- Create the Figure ---
link_colors_rgba = [f'rgba({int(c[1:3], 16)}, {int(c[3:5], 16)}, {int(c[5:7], 16)}, 0.6)' for c in links['color']]

fig = go.Figure(data=[go.Sankey(
    arrangement='snap',
    node=dict(
        pad=25,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=all_nodes,
        color='rgba(200, 200, 200, 0.8)' # Use a neutral color for nodes
    ),
    link=dict(
        source=links['source'],
        target=links['target'],
        value=links['value'],
        color=link_colors_rgba # Use the correctly formatted list of colors
    ))])



# --- Create Custom Legend ---
annotations = []
legend_y = 1.0
for _, row in df_chains.iterrows():
    annotations.append(dict(
        x=1.05,
        y=legend_y,
        xref='paper',
        yref='paper',
        text=f"<span style='color:{row['color']}; font-size: 18px;'>■</span> {row['chain_name']}",
        showarrow=False,
        align='left',
        xanchor='left',
        yanchor='top'
    ))
    legend_y -= 0.06 # Adjust vertical spacing

fig.update_layout(
    title_text="Exploit Chains: Visualizing Multi-Vulnerability Attack Paths",
    font_size=12,
    width=1200,
    height=700,
    margin=dict(l=20, r=250, b=50, t=100), # Increase right margin for legend
    annotations=annotations
)

fig.show()

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.colors

# Manually compiled data focusing only on multi-vulnerability attack chains.
# The 'vuln_entry' is the first flaw exploited, which then enables the 'vuln_payload'.
attack_chains_data = [
    {'chain_name': 'Euler Finance', 'vuln_entry': 'Access Control', 'vuln_payload': 'Business Logic', 'count': 1, 'loss': 197000000},
    {'chain_name': 'LI.FI (x2), Dexible', 'vuln_entry': 'Access / Input Validation', 'vuln_payload': 'Unsafe External Call', 'count': 3, 'loss': 11600000 * 2 + 2000000},
    {'chain_name': 'Hundred Finance, Abracadabra', 'vuln_entry': 'Price & Logic Flaw', 'vuln_payload': 'Rounding Error', 'count': 2, 'loss': 7400000 + 6500000},
    {'chain_name': 'Conic, EraLend, Sturdy', 'vuln_entry': 'Reentrancy', 'vuln_payload': 'Price Manipulation', 'count': 3, 'loss': 4200000 + 3400000 + 800000},
    {'chain_name': 'KiloEx, Jimbo\'s Protocol', 'vuln_entry': 'Access Control', 'vuln_payload': 'Price Manipulation', 'count': 2, 'loss': 7500000 * 2},
    {'chain_name': 'Onyx Protocol', 'vuln_entry': 'Price Manipulation', 'vuln_payload': 'Input Validation', 'count': 1, 'loss': 3800000},
    {'chain_name': 'Curio, OKX DEX', 'vuln_entry': 'Governance / Privilege Abuse', 'vuln_payload': 'Dangerous Delegatecall', 'count': 2, 'loss': 16000000 + 2700000},
    {'chain_name': 'Rho Market', 'vuln_entry': 'Privilege Abuse', 'vuln_payload': 'Price Manipulation', 'count': 1, 'loss': 7500000},
]

df_chains = pd.DataFrame(attack_chains_data)

# Define Loss Brackets
def get_loss_bracket(loss, count):
    avg_loss = loss / count
    if avg_loss > 50000000:
        return 'Major Loss (> $50M)'
    elif avg_loss > 10000000:
        return 'High Loss ($10M - $50M)'
    elif avg_loss > 1000000:
        return 'Medium Loss ($1M - $10M)'
    else:
        return 'Low Loss (< $1M)'

df_chains['Loss_Bracket'] = df_chains.apply(lambda row: get_loss_bracket(row['loss'], row['count']), axis=1)

# --- Prepare data for Sankey ---
color_palette = plotly.colors.qualitative.T10
df_chains['color'] = [color_palette[i % len(color_palette)] for i in range(len(df_chains))]

all_nodes = pd.concat([df_chains['vuln_entry'], df_chains['vuln_payload'], df_chains['Loss_Bracket']]).unique()
node_map = {node: i for i, node in enumerate(all_nodes)}

links = {'source': [], 'target': [], 'value': [], 'color': []}
for _, row in df_chains.iterrows():
    links['source'].append(node_map[row['vuln_entry']])
    links['target'].append(node_map[row['vuln_payload']])
    links['value'].append(row['count'])
    links['color'].append(row['color'])
    
    links['source'].append(node_map[row['vuln_payload']])
    links['target'].append(node_map[row['Loss_Bracket']])
    links['value'].append(row['count'])
    links['color'].append(row['color'])

# --- Create the Figure ---
link_colors_rgba = [f'rgba({int(c[1:3], 16)}, {int(c[3:5], 16)}, {int(c[5:7], 16)}, 0.7)' for c in links['color']]

fig = go.Figure(data=[go.Sankey(
    arrangement='snap',
    node=dict(
        pad=25,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=all_nodes,
        color='rgba(220, 220, 220, 0.9)'
    ),
    link=dict(
        source=links['source'],
        target=links['target'],
        value=links['value'],
        color=link_colors_rgba
    ))])

# --- Create Custom Legend ---
annotations = []
legend_y = 1.0
annotations.append(dict(x=1.03, y=legend_y + 0.03, xref='paper', yref='paper', text="<b>Exploit Chain Legend:</b>", showarrow=False, align='left', xanchor='left',yanchor='top', font=dict(size=14)))

for _, row in df_chains.iterrows():
    legend_text = (
        f"<span style='background-color:{row['color']}; border: 1px solid #555; border-radius: 4px; display: inline-block; width: 22px; height: 15px; vertical-align: middle; margin-right: 6px;'></span>"
        f"<span style='vertical-align: middle;'>{row['chain_name']}</span>"
    )
    annotations.append(dict(x=1.03, y=legend_y, xref='paper', yref='paper', text=legend_text, showarrow=False, align='left', xanchor='left', yanchor='top'))
    legend_y -= 0.08

fig.update_layout(
    title_text="Exploit Chains: Visualizing Multi-Vulnerability Attack Paths",
    font_size=13,
    width=1200,
    height=800,
    margin=dict(l=20, r=300, b=50, t=100),
    annotations=annotations,
    plot_bgcolor='white',
    paper_bgcolor='white'
)

fig.show()

In [ ]:
pip install --upgrade kaleido

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.colors
import plotly.io as pio

# Manually compiled data of multi-vulnerability attack chains
attack_chains_data = [
    {'chain_name': 'Euler Finance', 'vuln_entry': 'Access Control', 'vuln_payload': 'Business Logic', 'count': 1, 'loss': 197000000},
    {'chain_name': 'LI.FI, Dexible', 'vuln_entry': 'Input Validation', 'vuln_payload': 'Unsafe Call/Delegatecall', 'count': 3, 'loss': 11600000 * 2 + 2000000},
    {'chain_name': 'Hundred, Abracadabra', 'vuln_entry': 'Price/Logic', 'vuln_payload': 'Rounding Error', 'count': 2, 'loss': 7400000 + 6500000},
    {'chain_name': 'Conic, EraLend, Sturdy', 'vuln_entry': 'Reentrancy', 'vuln_payload': 'Price Manipulation', 'count': 3, 'loss': 4200000 + 3400000 + 800000},
    {'chain_name': 'KiloEx, Jimbo\'s', 'vuln_entry': 'Access Control', 'vuln_payload': 'Price Manipulation', 'count': 2, 'loss': 7500000 * 2},
    {'chain_name': 'Onyx Protocol', 'vuln_entry': 'Price Manipulation', 'vuln_payload': 'Input Validation', 'count': 1, 'loss': 3800000},
    {'chain_name': 'Curio, OKX DEX', 'vuln_entry': 'Governance', 'vuln_payload': 'Dangerous Delegatecall', 'count': 2, 'loss': 16000000 + 2700000},
    {'chain_name': 'Rho Market', 'vuln_entry': 'Privilege Abuse', 'vuln_payload': 'Price Manipulation', 'count': 1, 'loss': 7500000},
]

df_chains = pd.DataFrame(attack_chains_data)

# Assign loss brackets
def get_loss_bracket(loss, count):
    avg_loss = loss / count
    if avg_loss > 50000000:
        return 'Major Loss (> $50M)'
    elif avg_loss > 10000000:
        return 'High Loss ($10M - $50M)'
    elif avg_loss > 1000000:
        return 'Medium Loss ($1M - $10M)'
    else:
        return 'Low Loss (< $1M)'

df_chains['Loss_Bracket'] = df_chains.apply(lambda row: get_loss_bracket(row['loss'], row['count']), axis=1)

# Color and pattern simulation
color_palette = plotly.colors.qualitative.Plotly
patterns = ['solid', 'light', 'medium', 'dense', 'very_dense']
opacity_map = {
    'solid': 1.0,
    'light': 0.4,
    'medium': 0.6,
    'dense': 0.75,
    'very_dense': 0.9
}
df_chains['color'] = [color_palette[i % len(color_palette)] for i in range(len(df_chains))]
df_chains['pattern'] = [patterns[i % len(patterns)] for i in range(len(df_chains))]
df_chains['opacity'] = df_chains['pattern'].map(opacity_map)

# Build node mapping
all_nodes = pd.concat([df_chains['vuln_entry'], df_chains['vuln_payload'], df_chains['Loss_Bracket']]).unique()
node_map = {node: i for i, node in enumerate(all_nodes)}

# Create links
sources, targets, values, colors, labels = [], [], [], [], []

for _, row in df_chains.iterrows():
    rgba = lambda hex_color, opacity: f'rgba({int(hex_color[1:3], 16)}, {int(hex_color[3:5], 16)}, {int(hex_color[5:7], 16)}, {opacity})'
    color = rgba(row['color'], row['opacity'])

    # Entry → Payload
    sources.append(node_map[row['vuln_entry']])
    targets.append(node_map[row['vuln_payload']])
    values.append(row['count'])
    colors.append(color)
    labels.append(f"{row['count']} attack(s)")

    # Payload → Loss
    sources.append(node_map[row['vuln_payload']])
    targets.append(node_map[row['Loss_Bracket']])
    values.append(row['count'])
    colors.append(color)
    labels.append(f"{row['count']} attack(s)")

# Sankey diagram
fig = go.Figure(data=[go.Sankey(
    arrangement='snap',
    node=dict(
        pad=25,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=list(all_nodes),
        color='rgba(200, 200, 200, 0.8)'
    ),
    link=dict(
        source=sources,
        target=targets,
        value=values,
        color=colors,
        label=labels
    )
)])

# Add custom legend
annotations = []
legend_y = 1.0
for _, row in df_chains.iterrows():
    annotations.append(dict(
        x=1.05,
        y=legend_y,
        xref='paper',
        yref='paper',
        text=f"<span style='color:{row['color']}; font-size: 18px;'>■</span> {row['chain_name']} ({row['pattern']})",
        showarrow=False,
        align='left',
        xanchor='left',
        yanchor='top'
    ))
    legend_y -= 0.06

fig.update_layout(
    title_text="Exploit Chains: Visualizing Multi-Vulnerability Attack Paths",
    font_size=12,
    width=1200,
    height=700,
    margin=dict(l=20, r=280, b=50, t=100),
    annotations=annotations
)

# Export to PDF using kaleido
pio.write_image(fig, "exploit_chains.pdf", format='pdf', width=1200, height=700)

# Show the figure in the notebook/browser
fig.show()


In [28]:

import pandas as pd
import plotly.graph_objects as go
import plotly.colors

# Manually compiled data of multi-vulnerability attack chains.
attack_chains_data = [
    {'chain_name': 'Euler Finance', 'vuln_entry': 'Access Control', 'vuln_payload': 'Business Logic', 'count': 1, 'loss': 197000000},
    {'chain_name': 'LI.FI, Dexible', 'vuln_entry': 'Access / Input Validation', 'vuln_payload': 'Unsafe External Call', 'count': 3, 'loss': 11600000 * 2 + 2000000},
    {'chain_name': 'Hundred, Abracadabra', 'vuln_entry': 'Price & Logic Flaw', 'vuln_payload': 'Rounding Error', 'count': 2, 'loss': 7400000 + 6500000},
    {'chain_name': 'Conic, EraLend, Sturdy', 'vuln_entry': 'Reentrancy', 'vuln_payload': 'Price Manipulation', 'count': 3, 'loss': 4200000 + 3400000 + 800000},
    {'chain_name': 'KiloEx, Jimbo\'s', 'vuln_entry': 'Access Control', 'vuln_payload': 'Price Manipulation', 'count': 2, 'loss': 7500000 * 2},
    {'chain_name': 'Onyx Protocol', 'vuln_entry': 'Price Manipulation', 'vuln_payload': 'Input Validation', 'count': 1, 'loss': 3800000},
    {'chain_name': 'Curio, OKX DEX', 'vuln_entry': 'Governance / Privilege Abuse', 'vuln_payload': 'Dangerous Delegatecall', 'count': 2, 'loss': 16000000 + 2700000},
    {'chain_name': 'Rho Market', 'vuln_entry': 'Privilege Abuse', 'vuln_payload': 'Price Manipulation', 'count': 1, 'loss': 7500000},
]

df_chains = pd.DataFrame(attack_chains_data)

# Define Loss Brackets
def get_loss_bracket(loss, count):
    avg_loss = loss / count
    if avg_loss > 50000000:
        return 'Major Loss (> $50M)'
    elif avg_loss > 10000000:
        return 'High Loss ($10M - $50M)'
    elif avg_loss > 1000000:
        return 'Medium Loss ($1M - $10M)'
    else:
        return 'Low Loss (< $1M)'

df_chains['Loss_Bracket'] = df_chains.apply(lambda row: get_loss_bracket(row['loss'], row['count']), axis=1)

# --- Prepare data for Sankey ---
color_palette = plotly.colors.qualitative.T10
df_chains['color'] = [color_palette[i % len(color_palette)] for i in range(len(df_chains))]

all_nodes = pd.concat([df_chains['vuln_entry'], df_chains['vuln_payload'], df_chains['Loss_Bracket']]).unique()
node_map = {node: i for i, node in enumerate(all_nodes)}

links = {'source': [], 'target': [], 'value': [], 'color': []}
for _, row in df_chains.iterrows():
    links['source'].append(node_map[row['vuln_entry']])
    links['target'].append(node_map[row['vuln_payload']])
    links['value'].append(row['count'])
    links['color'].append(row['color'])
    
    links['source'].append(node_map[row['vuln_payload']])
    links['target'].append(node_map[row['Loss_Bracket']])
    links['value'].append(row['count'])
    links['color'].append(row['color'])

# --- Create the Figure ---
link_colors_rgba = [f'rgba({int(c[1:3], 16)}, {int(c[3:5], 16)}, {int(c[5:7], 16)}, 0.7)' for c in links['color']]

fig = go.Figure(data=[go.Sankey(
    arrangement='snap',
    node=dict(
        pad=25,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=all_nodes,
        color='rgba(220, 220, 220, 0.9)'
    ),
    link=dict(
        source=links['source'],
        target=links['target'],
        value=links['value'],
        color=link_colors_rgba
    ),
    textfont=dict(color='black', size=15)  # ✅ Correct placement
)],
    )

# Add a manual legend mapping each color to the chain_name
for _, row in df_chains.iterrows():
    fig.add_trace(go.Scatter(
        x=[None], y=[None],
        mode='markers',
        marker=dict(size=12, color=row['color']),
        legendgroup=row['chain_name'],
        showlegend=True,
        name=row['chain_name']
    ))

fig.update_layout(
    font_size=13,
    width=1100,
    height=700,
    plot_bgcolor='white',
    paper_bgcolor='white',
    legend=dict(
        title="Incident Chains",
        orientation="v",
        yanchor="top",
        y=1.0,
        xanchor="right",
        x=1.26,
        font=dict(size=15, color='black'),  # Increased size and darker font
        title_font=dict(size=15, color='black'),  # Optional: style title separately
        bgcolor='rgba(255,255,255,0.8)'
    ),
    xaxis=dict(showticklabels=False, showgrid=False, zeroline=False),
    yaxis=dict(showticklabels=False, showgrid=False, zeroline=False),
    margin=dict(l=10, r=10, t=10, b=10),  # left, right, top, bottom
)

import plotly.io as pio

# Save as PDF
#pio.write_image(fig, "exploit_chains_sankey.pdf", format='pdf', width=1100, height=700)

pio.write_image(
    fig,
    "exploit_chains_sankey.pdf",
    format="pdf",
    width=1100,  # or increase to 1600 for journal quality
    height=600,
    scale=1  # keep scale=1 for vector PDF
)

fig.show()